In [2]:
import gym
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
import time, math, random
from typing import Tuple
import tqdm
import matplotlib.pyplot as plt

## Q-Learning

#### q(s,a) = q(sa-a) + lr(r+ymaxq(s',a')) ymax: Discount factor (0,1) a': Future action s': Future state


In [3]:
environment = gym.make('CartPole-v1')

#Q-table
#Actions: left - right
print("Action space: ", environment.action_space.n)

[2022-10-18 10:48:52,869] Making new env: CartPole-v1


Action space:  2


/home/karoline/OneDrive/Emner/Semester 5/Maskinlæring/Exercise7/reinf-env/lib/python3.10/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
class CartPoleAgent:
    def __init__(self, env, episodes):
        self.env = env
        # One table: actions * possible angles, second: actions*pole_vels
        self.n_bins = (6, 12)
        self.q_table = np.zeros(self.n_bins + (env.action_space.n, ))
        self.lower_bounds = [env.observation_space.low[2], -math.radians(50)]
        self.upper_bounds = [env.observation_space.high[2], math.radians(50)]
        self.lr = 0.1
        self.min_lr_rate = 0.1
        self.min_explo_rate=0.1
        self.decay = 24
        self.discount = 1.0
        self.episodes = episodes

    #Gets dsicrete value since the original state is contionus (time)
    def get_discrete(self, state):
        _, __, angle, pole_vel = state
        est = KBinsDiscretizer(n_bins=self.n_bins, encode='ordinal', strategy='uniform')
        est.fit([self.lower_bounds, self.upper_bounds])
        return tuple(map(int, est.transform([[angle, pole_vel]])[0]))
    # Returns the policy used for getting the next action (highest value in q table next)
    def get_policy(self, state):
        return np.argmax(self.q_table[state])

    #Q function used to update the prev q-value
    def update_q_value(self, reward, state, action, new_state):
        return self.lr * (reward + self.discount * np.max(self.q_table[new_state]) - self.q_table[state][action])

    #Adaptive lr. Used to adapt the learning rate
    def get_lr(self, n):
        return max(self.min_lr_rate, min(1.0, 1.0 - math.log10((n+1) / self.decay)))

    #Decaying explo rate/epsilon
    def get_explo_rate(self, n):
        return max(self.min_explo_rate, min(1.0, 1.0 - math.log10((n+1) / self.decay)))

    def train(self):
        scores = []
        for e in tqdm.tqdm(range(self.episodes)):
            current_state = self.get_discrete(self.env.reset())
            self.lr = self.get_lr(e)
            done = False
            #Tracks how many inputs it survives
            score = 0
            while not done:
                action = self.get_policy(current_state)

                #Random action (exploration). Every x amount of times the agent should do a random action
                if np.random.random() <= self.get_explo_rate(e):
                    action = self.env.action_space.sample()

                #After each step an observable state, reward and if it is done is returned
                obs, reward, done, _ = self.env.step(action)
                #Discretes the new state
                new_state = self.get_discrete(obs)

                #Gets new q-value
                #learnt_value = self.update_q_value(reward, current_state, action, new_state)
                self.q_table[current_state][action] += self.update_q_value(reward, current_state, action, new_state)
                #Gets old val from q-table

                current_state = new_state
                #For each loop it statys above 12 degs, it adds to the score
                score += 1

            scores.append(score)

        #print(scores)
        print("Training done!")
        return scores

In [6]:
import warnings
warnings.filterwarnings('ignore')
model = CartPoleAgent(environment, episodes=2000)
scores = model.train()

print(scores)
print(model.get_discrete_state())



 43%|████▎     | 856/2000 [00:53<01:36, 11.91it/s]